In [1]:
from pathlib import Path
import re
import pandas as pd
import requests
import time
import datetime
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [2]:
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / 'data' # same as os.path.join(BASE_DIR, 'data')
if not DATA_DIR.exists():
    DATA_DIR.mkdir(exist_ok=True)

product_category_links_output = DATA_DIR / 'category-products.csv'
product_output = DATA_DIR / 'products.csv'

In [3]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [4]:
categories = [
    {"name": "appliances", "url": 'https://www.amazon.com/Best-Sellers-Appliances/zgbs/appliances/'},
    {"name": "electronics", "url": 'https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/'},
    {"name": "books", "url": 'https://www.amazon.com/best-sellers-books-Amazon/zgbs/books/'}
]


In [5]:
regex_options = [
    r"https://www.amazon.com/gp/product/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/dp/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/",
]

def extract_product_id_from_url(url):
    product_id = None
    for regex_str in regex_options:
        regex = re.compile(regex_str)
        match = regex.match(url)
        if match != None:
            try:
                product_id = match['product_id']
            except:
                pass
    return product_id

In [6]:
def clean_page_links(page_links=[], category=None):
    final_page_links = []
    for url in page_links:
        product_id = extract_product_id_from_url(url)
        if product_id != None:
            final_page_links.append({"url": url, "product_id": product_id, "category": category})
    return final_page_links

In [7]:
def scrape_category_product_links(categories=[]):
    all_product_links = []
    for category in categories:
        time.sleep(1.5)
        url = category.get('url')
        driver.get(url)
        body_el = driver.find_element(By.CSS_SELECTOR, "body")
        html_str = body_el.get_attribute('innerHTML')
        html_obj = HTML(html=html_str)
        page_links = [f'https://www.amazon.com{x}' for x in html_obj.links if x.startswith('/')]
        cleaned_links = clean_page_links(page_links=page_links, category=category)
        all_product_links += cleaned_links
    return all_product_links



In [8]:
def extract_categories_and_save(categories=[]):
    all_product_links = scrape_category_product_links(categories)
    category_df = pd.DataFrame(all_product_links)
    category_df.to_csv(product_category_links_output, index=False)

In [9]:
extract_categories_and_save(categories=categories)

In [10]:
def scrape_product_page(url, title_lookup = '#productTitle', price_lookup = '.a-price-whole'):
    driver.get(url)
    time.sleep(.5)
    body_el = driver.find_element(By.CSS_SELECTOR, "body")
    html_str = body_el.get_attribute('innerHTML')
    html_obj = HTML(html=html_str)
    product_title = html_obj.find(title_lookup, first=True).text
    product_price = html_obj.find(price_lookup, first=True).text
    return product_title, product_price

In [11]:
def perform_scrape(cleaned_items=[]):
    data_extracted = []
    for obj in cleaned_items:
        link = obj['url']
        product_id = obj['product_id']
        title, price = (None, None)
        try:
            title, price = scrape_product_page(link)
        except:
            pass
        if title != None and price != None:
            print(link, title, price)
        product_data = {'url': link, 'product_id': product_id, 'title': title, 'price': price}
        data_extracted.append(product_data)
    return data_extracted

In [12]:
# extracted_data = perform_scrape(cleaned_items=cleaned_links)

In [13]:
# print(extracted_data)

In [14]:
def row_scrape_event(row, *args, **kwargs):
    link = row['url']
    scraped = 0
    try:
        scraped = row['scraped']
    except:
        pass
    if scraped == 1 or scraped == '1':
        return row
    product_id = row['product_id']
    title, price = (None, None)
    try:
        title, price = scrape_product_page(link)
    except:
        pass
    row['title'] = title
    row['price'] = price
    row['scraped'] = 1
    row['timestamp'] = datetime.datetime.now().timestamp()
    return row

In [15]:
df = pd.read_csv(product_category_links_output)
df.head()

,url,product_id,category
0,https://www.amazon.com/VIVOHOME-Electric-Porta...,B07T1XPVX3,"{'name': 'appliances', 'url': 'https://www.ama..."
1,https://www.amazon.com/hOmeLabs-Portable-Maker...,B071J2LSQS,"{'name': 'appliances', 'url': 'https://www.ama..."
2,https://www.amazon.com/GE-Profile-Countertop-N...,B07YF9SGBW,"{'name': 'appliances', 'url': 'https://www.ama..."
3,https://www.amazon.com/Frigidaire-Countertop-M...,B07J1YVSNQ,"{'name': 'appliances', 'url': 'https://www.ama..."
4,https://www.amazon.com/Countertop-Portable-Sel...,B09MJWFL31,"{'name': 'appliances', 'url': 'https://www.ama..."


In [16]:
df.shape

(89, 3)

In [17]:
df_sub = df.copy() #head(n=10)

In [18]:
df_sub = df_sub.apply(row_scrape_event, axis=1)

In [19]:
# df.to_csv(product_output, index=False)

In [20]:
products_df = pd.read_csv(product_output)
products_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\rober\\coding\\api_projects\\30Days\\Day 18\\Notebooks\\data\\products.csv'

In [ ]:
final_df = pd.concat([products_df, df_sub])
final_df.to_csv(product_output, index=False)

In [ ]:
df_sub